In [1]:

import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import re
from re import split

In [2]:
def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

mlen = 3 # number states
tlen = 3 # # ways to count time
excludedvids=[]
plas_old = [1/0.48, 1/0.43]
glas_old = [1/4, 1/3]

In [3]:
def parser_arr(filename):
    # returns array of all traansitions and list of all class and transition thresholds
    transitions = []
    thresholds = []
    vidarrs = []
    timestotrans = []
    statelists = []
    with open(filename) as textfile:
        line = textfile.readline()
        #print('opened', line)
        while line:
            if 'Processing' in line:
                vidnum = line.split('pp_')[-1]
                vidnum = vidnum.split('.avi')[0]
                vidnum = vidnum.split('_')[0]
                #print(vidnum)
                vidnum = int(vidnum)
                line = textfile.readline()
            if 'T threshold'in line:
                thresh_set = []
                for word in line.split():
                    try:
                        thresh_set.append(float(word)) #collect T_treshold
                    except ValueError:
                        pass
                    
                line = textfile.readline()
                #print('after t', line)
                if 'C threshold' in line:
                    for word in line.split():
                        try:
                            thresh_set.append(float(word)) # colect C_threshold
                        except ValueError:
                            pass
                                          
                    #Initialise collecction arrays for this threshold pair
                    if len(thresholds) ==0 or thresholds[-1] != thresh_set:
                        #print(thresh_set)
                        thresholds.append(thresh_set)
                        #vidnums=[]
                        vidarrs.append([[] for i in range(3)])
                        vidsarr = np.zeros((mlen,mlen))
                        transitions.append(vidsarr)
                        #timestotrans.append(np.zeros((tlen,mlen,mlen)))
                        statelists.append([])
                    else:
                        pass    
                    
                    # Collect transition data
                    line = textfile.readline()
                    #print('array lives here', line)
                    trarr = read_square_arr(line, textfile)
                    transitions[-1] += trarr
                    vidarrs[-1][0].append(trarr)
                    vidarrs[-1][1].append(vidnum)
        
                    # Collect time data
                    #print(len(timestotrans[-1]))
                    tiarrs = []
                    for i in range(tlen):
                        line = textfile.readline()
                        #print('array lives there', line)
                        tiarr = read_square_arr_list(line, textfile) 
                        #print(arr)
                        #times += tiarr   
                        #print(times)
                        tiarrs.append(tiarr)
                        line = textfile.readline()
                        print('arr','\n', tiarr)
                    #print("collected 3 rrays: ", tiarrs)
                    vidarrs[-1][2].append(tiarrs)                       
                    # Collect state data 
                    statelists[-1].append([float(f) for f in filter(int_from_text, split(' |\[|]', line))])
                    #print(thresholds[-1])
                    #print(thresholds[-1])
                else:
                    print("missing C thresh")
                
                
                    
            line = textfile.readline()
            #print(line, 'moved on')
                    
                
            
    textfile.close()
    #print(transitions, thresholds)
    return([np.array(transitions), np.array(thresholds), np.array(vidarrs), np.array(timestotrans), statelists]) 

def int_from_text(text):
    try:
        float(text)
        return True
    except:
        return False
    
def int_from_text_list(textlist):
    #print('textlist: ', textlist)
    elements = []
    for el in textlist:
        nums = [int(s) for s in  re.findall(r'\d+',el)]
        elements.append(nums)
        #print("nums for element ",el,": ", nums )
    #print('elements total: ', elements)
    return elements
    
def read_square_arr(line, textfile):
    if '[[' in line:
        arr = np.zeros((mlen,mlen))
        #print(split(' |\[|]', line))
        linenums = [float(f) for f in filter(int_from_text, split(' |\[|]', line))]
        #print(linenums)
        arr[0,:] = linenums

        line = textfile.readline()
        #print('array lives here', line)           
        linenums = [float(f) for f in filter(int_from_text, split(' |\[|]', line))]
        #print(linenums)
        arr[1,:] = linenums

        line = textfile.readline()
        #print('array lives here', line) 
        linenums = [float(f) for f in filter(int_from_text, split(' |\[|]', line))]
        #print(linenums)
        arr[2,:] = linenums

        return(arr)
        #print(arr,'\n', transitions[-1])
    else:
        print("missing trans array")
sttr = '\[*list'
def read_square_arr_list(line, textfile):
    #print('square array start: ', line)
    if '[[' in line:
        arr =[]
        
        while ')]' not in line:
            line2 = textfile.readline()
            line += line2
            #print('LINE', line)           
            
        prelims = [x for x in split(sttr , line.replace('\n', '')) if re.match('\S', x)]
        #print(prelims)
        linenums = int_from_text_list(prelims)
        #print(linenums)
        arr.append(linenums)

        line = textfile.readline()        
        while ')]' not in line:
            line2 = textfile.readline()
            line += line2
            #print('LINE', line)   
            
        prelims = [x for x in split(sttr , line.replace('\n', '')) if re.match('\S', x)]
        #print(prelims)
        linenums = int_from_text_list(prelims)
        #print(linenums)
        arr.append(linenums)

        line = textfile.readline()
        while ')]' not in line:
            line2 = textfile.readline()
            line += line2
            #print('LINE', line)   
        
        prelims = [x for x in split(sttr , line.replace('\n', '')) if re.match('\S', x)]
        #print(prelims)
        linenums = int_from_text_list(prelims)
        #print(linenums)
        arr.append(linenums)
        #print("array", arr)

        return(np.array(arr))
        #print(arr,'\n')
    else:
        print("missing times array")


        
def ratio(row):
    sums = np.sum(row)
    if sums > 0:
        return row/sums
    else:
        return row
    
def col_norm(array):
    cols = array.T
    cols = np.array([ratio(j)*100 for j in cols])
    return cols.T
def row_norm(array):
    rows = np.array([ratio(i)*100 for i in array])
    return rows

In [4]:

cols = [ 'mediumslateblue','fuchsia','darkviolet' ] #p, g, b
labels = ["Plastic only touches", "Glass only touches", "Both touch"]

In [5]:
trans_runs= parser_arr(glob('./wash*both.txt')[0]) # the output file with washed styrene information
print([len(i) for i in trans_runs]) # should be the same length = the number of runs
test = trans_runs[1][0]
pprint.pprint(test[0]) # every video array for the first run 


arr 
 [[list([1023, 773, 626, 1079, 915, 372, 374, 443, 819, 161, 231, 445, 460, 1590, 715, 441, 2694, 1009, 259, 391, 900, 289, 554, 819, 255, 448, 821, 547, 461, 81, 272, 482, 2782])
  list([])
  list([643, 1919, 2332, 1576, 486, 362, 253, 231, 963, 4269, 529, 363, 1355])]
 [list([]) list([]) list([])]
 [list([1405, 125, 2204, 1496, 2153, 49, 506, 298, 417, 973, 1219, 2434, 821, 352])
  list([])
  list([344, 303, 446, 171, 367, 1907, 627, 237, 345, 479, 181, 224, 1430, 755])]]
arr 
 [[list([131, 290, 483, 623, 194, 177, 142, 76, 304, 13, 8, 54, 78, 205, 175, 153, 92, 644, 3, 189, 405, 0, 242, 455, 86, 281, 708, 244, 21, 0, 78, 315, 123])
  list([])
  list([22, 310, 19, 1094, 366, 142, 17, 119, 777, 1136, 91, 40, 358])]
 [list([]) list([]) list([])]
 [list([1, 2, 4, 27, 5, 3, 289, 62, 15, 3, 0, 158, 12, 104]) list([])
  list([221, 18, 61, 0, 5, 1, 6, 23, 1, 9, 4, 14, 37, 170])]]
arr 
 [[list([892, 483, 143, 456, 721, 195, 232, 367, 515, 148, 223, 391, 382, 1385, 540, 288, 2602, 365, 2

NameError: name 'pprint' is not defined

In [ ]:
trans_runs_old= parser_arr_old(glob('./output wash*halftrimmeddebugg.txt')[0]) # the output file with washed styrene information
print([len(i) for i in trans_runs_old]) # should be the same length = the number of runs
test = trans_runs_old[1][0] 
pprint.pprint(test[0]) # Every video array for the first run


In [ ]:
expt_list = [trans_runs,trans_runs_old]
exptnames = ["3 Ethylene, 1 Styrene - 2nd", "1st ethylene-1-styrene"]
vidstoexclude = [excludedvids, excludedvids]
criteria = []  # unclear what value this is
stored_y_values = {} # the transition matrices, by 
times = {}
c_threshes = {exptname: [] for exptname in exptnames} 
t_threshes = {exptname: [] for exptname in exptnames} 

for expt, exptname, excludedvids in zip(expt_list, exptnames, vidstoexclude):   
    print(exptname)
    y_vals = {}
    t_vals = {}
    run_indices = list(range(len(expt[0]))) # for all the videos in trans_runs
    for runindex in run_indices:
        trans_threshes = expt[0][runindex]
        t_threshes[exptname].append(trans_threshes[0])
        c_threshes[exptname].append(trans_threshes[1])
        criteria.append(str(int(trans_threshes[0]))+','+str(int(trans_threshes[1])))

        vidarrs = np.array(expt[1][runindex][0]) 
        vidnums = expt[1][runindex][1]
        print("Run number {}, thresholds {}".format(runindex, criteria[-1]))
        print('videos recorded', vidnums)

        allowed_inds = [x for x in range(len(vidnums)) if vidnums[x] not in excludedvids] # index of good videos in vidnums list
        vidnums = [vidnums[x] for x in allowed_inds]
        vidarrs = vidarrs[allowed_inds]   # Choosing arrays in vidarrs corresponding to correct videos from vidnums        
        if len(expt[1][runindex]) > 2:
            timarrs = np.array(expt[1][runindex][2]) 
            timarrs = timarrs[allowed_inds]
        else:
            timarrs = []            
        print('videos used in calculations', vidnums)

        if len(vidarrs) == len(vidnums):
            vidindices = list(range(len(vidarrs))) # need to iterate over every video that's good
        else:
            print("Something's wrong with the indexing!")

        # collect list of probability transitions; one per video    
        percent_pairs = {}
        transarrlists = {}
        for vidind in vidindices: 
            T_obb = vidarrs[vidind] # observed transition matrix for this video
            print('\nMatrix observed for video {}: \n'.format(vidind), T_obb)
            T_obb_norm = np.vstack([ratio(T_obb[i]) for i in range(len(T_obb))]) # normailise each row s othe transitions sm to 1
            counts =  ratio(np.sum(T_obb_norm, axis=0))*100 # convert to percentages
            print('Precentages:', counts)
            percent_pairs[vidind] = counts
            if len(timarrs) != 0:
                Ti_obb = timarrs[vidind]
                print("Times:", Ti_obb)
                transarrlists[vidind] =  Ti_obb
        y_vals[runindex]= percent_pairs
        if bool(transarrlists):
            t_vals[runindex] = transarrlists
        print('\nTotal transitions: ',np.sum(vidarrs), "\n Sum of all transition matrices:\n", np.sum(vidarrs, axis=0), '\n\n') 
    stored_y_values[exptname] = y_vals
    if bool(t_vals):
        times[exptname] = t_vals
#stored_y_values = np.array(stored_y_values)

    

In [ ]:
exptnum = 0 # to to separate the labeling for the runs
bignum = 100 # to separate the labeling for the runs
vidprops = {}

# collect, for each video (in each run), the list of matrices recorded
for ind, expt in enumerate(stored_y_values.keys()):
    exptdicts = stored_y_values[expt] # 3 dicts, one per run
    for runnum, rundict in exptdicts.items():
        keys = rundict.keys() # vifdeo number of run
        for key in keys: # a video number
            vidnumkey = exptnum + key
            if vidnumkey not in vidprops.keys():
                vidprops[vidnumkey] = rundict[key]                
            else:
                vidprops[vidnumkey] = np.vstack((vidprops[vidnumkey], rundict[key]))
    exptnum = exptnum + bignum

#Got to double check!
keys = vidprops.keys()
vidsums = [] #sum all the probabilities 
for key in keys:
    allprobs = vidprops[key]
    print("Collected all probabilities for video  {}: \n{}".format(key, allprobs))
    if len(np.shape(allprobs)) > 1:
        allprobs_sum = np.sum(allprobs, axis = 0) / len(allprobs)
    else:
        allprobs_sum = allprobs
    vidsums.append(allprobs_sum )
    print("And summed and averaged all probabilities:\n{}\n".format(allprobs_sum))
vidsums = np.array(vidsums)
vidinds = list(range(len(keys)))    
print("\nShape of total data array", np.shape(vidsums))
    
fig,ax = plt.subplots()
print('\n',len(vidnums), len(y1))   
lines = [y1,y2,y3]
for i in range(len(lines)):
    ax.plot(vidnums, lines[i], c = cols[i], ls = '-', label = labels[i], marker = "^")
    plt.axhline(y = max(lines[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(lines[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'center right', framealpha = 0.3)
plt.xlabel('video number', fontsize=18)
plt.ylabel('observed probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for each video")

    
    
#Print probabilities by v threshold
    
    
    
# Printprobabilities by video

In [ ]:
# Print probabilitie sby video
plotter(vidinds, vidsums.T, cols, 'Vidnums')


In [ ]:
index = list(range(len(trans_runs[0])))

criteria = []
#t_thresh_list = [20, 33, 46, 60]
stored_y_values = []
c_threshes = []
t_threshes = []
for i in index:
    x1 = []
    x2 = []
    x3 = []
    trans_matrices = []
    N_transitions = trans_runs[0][i]
    trans_threshes = trans_runs[1][i]
    vidarrs = np.array(trans_runs[2][i][0])
    vidnums = trans_runs[2][i][1]
    print('vidnums', vidnums)
    allowed_inds = [x for x in range(len(vidnums)) if vidnums[x] not in excludedvids]
    print('disallowed numbers', excludedvids)
    print('disallowed indices', [x for x in range(len(vidnums)) if vidnums[x] in excludedvids])
    print('allowed indices', allowed_inds)
    vidnums = [vidnums[x] for x in allowed_inds]
    print('vidnums filtered', vidnums)
    vidarrs = vidarrs[allowed_inds]   
    vidindices = list(range(len(vidarrs)))
    for vidind in vidindices: 
        T_obb = vidarrs[vidind]
        trans_matrices.append(T_obb)
        T_obb_norm = np.vstack([ratio(T_obb[i]) for i in range(len(T_obb))])
        #print(T_obb, '\n', np.around(T_obb_norm, 2), np.sum(T_obb_norm, axis=1))
        eigvals, eigvecs = np.linalg.eig(T_obb_norm.T)
        eig_one_index = [x for x in range(len(eigvals)) if eigvals[x]==max(eigvals)][0]
        eigvec_one = eigvecs[:, eig_one_index]
        evec_one = eigvec_one*100 / np.sum(eigvec_one)
        counts =  ratio(np.sum(T_obb_norm, axis=0))*100#/np.sum(T_obb_norm)
        #print('\n y values incoming from e_vector\n',evec_one, np.sum(T_obb_norm, axis=0)*100/np.sum(T_obb_norm), 
              #np.sum(T_obb, axis=0))
        x1.append(counts[0])
        x2.append(counts[1])
        x3.append(counts[2])
    if len(x1) == len(vidnums):
        #vidarrs = np.cumsum(vidarrs, axis=0)
        t_threshes.append(trans_threshes[0])
        c_threshes.append(trans_threshes[1])
        criteria.append(str(int(trans_threshes[0]))+','+str(int(trans_threshes[1])))
        stored_y_values.append([x1,x2,x3])
#print(t_threshes,c_threshes)    

# Print probabilitie sby c threshold
stored_y_values = np.array(stored_y_values)
cs = list(set(c_threshes))
list.sort(cs)
#print('\n', stored_y_values, '\n', c_threshes, '\n', cs, '\n')
y1 = []
y2 = []
y3 = []
for j in cs:
    inds = np.where(np.array(c_threshes) == j)
    y_values_selected = stored_y_values[inds] # matrix where each row is a t,c that includes the particular c threshold as the c
    #print('\noriginal y values\n', y_values_selected)
    y_values_selected = np.sum(y_values_selected, axis=0)
    #print('\nsummed y values\n',j, y_values_selected)    
    collected_value_across_all_vids = ratio([sum(i) for i in y_values_selected])*100
    #y1,y2,y3 = y_values_selected # one y value per video, and all only for this c threshold, summed over the t thresholds used
    #print(collected_value_across_all_vids, sum(collected_value_across_all_vids))
    y1.append(collected_value_across_all_vids[0])
    y2.append(collected_value_across_all_vids[1])
    y3.append(collected_value_across_all_vids[2])
print(cs,y1,y2,y3)
fig,ax = plt.subplots()
print(len(cs), len(y1))        
lines = [y1,y2,y3]
for i in range(len(lines)):
    ax.plot(cs, lines[i], c = cols[i], ls = '-', label = labels[i], marker="^")
    plt.axhline(y = max(lines[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(lines[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'upper right', framealpha = 0.3)
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.xlabel('Stability threshold', fontsize=18)
plt.ylabel('observed probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for each value of the stability threshold")
    
#Print probabilities by v threshold
    
    
    
# Printprobabilities by video

In [ ]:
index = list(range(len(trans_runs[0])))

criteria = []
#t_thresh_list = [20, 33, 46, 60]
stored_y_values = []
c_threshes = []
t_threshes = []
for i in index:
    x1 = []
    x2 = []
    x3 = []
    trans_matrices = []
    N_transitions = trans_runs[0][i]
    trans_threshes = trans_runs[1][i]
    vidarrs = np.array(trans_runs[2][i][0])
    vidnums = trans_runs[2][i][1]
    print('vidnums', vidnums)
    allowed_inds = [x for x in range(len(vidnums)) if vidnums[x] not in excludedvids]
    print('disallowed numbers', excludedvids)
    print('disallowed indices', [x for x in range(len(vidnums)) if vidnums[x] in excludedvids])
    print('allowed indices', allowed_inds)
    vidnums = [vidnums[x] for x in allowed_inds]
    print('vidnums filtered', vidnums)
    vidarrs = vidarrs[allowed_inds]   
    vidindices = list(range(len(vidarrs)))
    for vidind in vidindices: 
        T_obb = vidarrs[vidind]
        trans_matrices.append(T_obb)
        T_obb_norm = np.vstack([ratio(T_obb[i]) for i in range(len(T_obb))])
        #print(T_obb, '\n', np.around(T_obb_norm, 2), np.sum(T_obb_norm, axis=1))
        eigvals, eigvecs = np.linalg.eig(T_obb_norm.T)
        eig_one_index = [x for x in range(len(eigvals)) if eigvals[x]==max(eigvals)][0]
        eigvec_one = eigvecs[:, eig_one_index]
        evec_one = eigvec_one*100 / np.sum(eigvec_one)
        counts =  ratio(np.sum(T_obb_norm, axis=0))*100#/np.sum(T_obb_norm)
        #print('\n y values incoming from e_vector\n',evec_one, np.sum(T_obb_norm, axis=0)*100/np.sum(T_obb_norm), 
              #np.sum(T_obb, axis=0))
        x1.append(counts[0])
        x2.append(counts[1])
        x3.append(counts[2])
    if len(x1) == len(vidnums):
        t_threshes.append(trans_threshes[0])
        c_threshes.append(trans_threshes[1])
        criteria.append(str(int(trans_threshes[0]))+','+str(int(trans_threshes[1])))
        stored_y_values.append([x1,x2,x3])
#print(t_threshes,c_threshes)    

# Print probabilitie sby t threshold
stored_y_values = np.array(stored_y_values)
ts = list(set(t_threshes))
list.sort(ts)
#print('\n', stored_y_values, '\n', c_threshes, '\n', cs, '\n')
y1 = []
y2 = []
y3 = []
for j in ts:
    inds = np.where(np.array(t_threshes) == j)
    y_values_selected = stored_y_values[inds] # matrix where each row is a t,c that includes the particular c threshold as the c
    #print('\noriginal y values\n', y_values_selected)
    y_values_selected = np.sum(y_values_selected, axis=0)
    #print('\nsummed y values\n',j, y_values_selected)    
    collected_value_across_all_vids = ratio([sum(i) for i in y_values_selected])*100
    #y1,y2,y3 = y_values_selected # one y value per video, and all only for this c threshold, summed over the t thresholds used
    #print(collected_value_across_all_vids, sum(collected_value_across_all_vids))
    y1.append(collected_value_across_all_vids[0])
    y2.append(collected_value_across_all_vids[1])
    y3.append(collected_value_across_all_vids[2])
print(ts,y1,y2,y3)
fig,ax = plt.subplots()
print(len(ts), len(y1))        
lines = [y1,y2,y3]
for i in range(len(lines)):
    ax.plot(ts, lines[i], c = cols[i], ls = '-', label = labels[i], marker="^")
    plt.axhline(y = max(lines[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(lines[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'upper right', framealpha = 0.3)
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.xlabel('Transition threshold', fontsize=18)
plt.ylabel('observed probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for each value of the stability threshold")
    
    
#Print probabilities by v threshold
    
    
    
# Printprobabilities by video

In [ ]:
index = list(range(len(trans_runs[0])))
def row_norm(array):
    rows = np.array([ratio(j)*100 for j in array])
    return rows
criteria = []
#t_thresh_list = [20, 33, 46, 60]
stored_y_values = []
c_threshes = []
t_threshes = []
vidnumset = []
print('\nlen indices', len(index))
for i in index:
    x1 = []
    x2 = []
    x3 = []
    trans_matrices = []
    N_transitions = trans_runs[0][i]
    trans_threshes = trans_runs[1][i]
    if True:#len(trans_runs[2][i][0]) == len(vidarrs):
        vidarrs = np.array(trans_runs[2][i][0])
        vidnums = trans_runs[2][i][1]
        print('vidnums', vidnums)
        allowed_inds = [x for x in range(len(vidnums)) if vidnums[x] not in excludedvids]
        print('disallowed numbers', excludedvids)
        print('disallowed indices', [x for x in range(len(vidnums)) if vidnums[x] in excludedvids])
        print('allowed indices', allowed_inds)
        vidnums = [vidnums[x] for x in allowed_inds]
        print('vidnums filtered', vidnums)
        vidarrs = vidarrs[allowed_inds]   
        vidindices = list(range(len(vidarrs)))
        for vidind in vidindices: 
            T_obb = vidarrs[vidind]
            trans_matrices.append(T_obb)
            T_obb_norm = np.vstack([ratio(T_obb[i]) for i in range(len(T_obb))])
            #print(T_obb, '\n', np.around(T_obb_norm, 2), np.sum(T_obb_norm, axis=1))
            eigvals, eigvecs = np.linalg.eig(T_obb_norm.T)
            eig_one_index = [x for x in range(len(eigvals)) if eigvals[x]==max(eigvals)][0]
            eigvec_one = eigvecs[:, eig_one_index]
            evec_one = eigvec_one*100 / np.sum(eigvec_one)
            counts =  ratio(np.sum(T_obb_norm, axis=0))*100#/np.sum(T_obb_norm)
            #print('counts', counts)
            #print('\n y values incoming from e_vector\n',evec_one, np.sum(T_obb_norm, axis=0)*100/np.sum(T_obb_norm), 
                  #np.sum(T_obb, axis=0))
            x1.append(counts[0])
            x2.append(counts[1])
            x3.append(counts[2])
        t_threshes.append(trans_threshes[0])
        c_threshes.append(trans_threshes[1])
        criteria.append(str(int(trans_threshes[0]))+','+str(int(trans_threshes[1])))
        stored_y_values.append([x1,x2,x3])#,x3])
#print(t_threshes,c_threshes)    
print('criteria', criteria)
# Print probabilitie sby video
#stored_y_values = [stored_y_values[x] for x in range(len(stored_y_values))]
stored_y_values = [z for z in stored_y_values if len(z[0]) == len(stored_y_values[0][0])] #np.array(stored_y_values)
print('\nstored_y_values',stored_y_values,'\n', len(stored_y_values), len(stored_y_values[0]), len(stored_y_values[0][1]),stored_y_values[0][1])
y_values_selected = np.array(np.sum(stored_y_values, axis=2))
print(y_values_selected)
y_values_selected = row_norm(y_values_selected)
print('\n','\nsummed y values\n', y_values_selected, y_values_selected.T,'\n', len(y_values_selected), len(y_values_selected[0]))
y1,y2,y3 = y_values_selected.T 

fig,ax = plt.subplots()  
ts = list(range(len(y1)))
print(len(ts), len(y1))        
lines = [y1,y2,y3]
for i in range(len(lines)):
    ax.plot(ts, lines[i], c = cols[i], ls = '-', label = labels[i], marker="^")
    plt.axhline(y = max(lines[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(lines[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'upper right', framealpha = 0.3)
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
plt.ylabel('observed probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for each value of the stability threshold")
plt.xlabel('Indices', fontsize=18)


In [ ]:
v1 = y1
v2 = y2
v3=y3

In [ ]:
"tESTING HYPOTHESIS WITH all position, early state important, end sate proportional to bond strengths"
#narrow observations array
alphas = np.linspace(0, 20, 100,endpoint =False)
betas = np.linspace(0, 20, 100,endpoint =False)
x1 = []
x2 = []
x3 = []
for aln in range(len(alphas)):
    a = []
    b = []
    c = []
    i = alphas[aln]
    if i ==1:
        alpha1 = aln
    for bln in range(len(betas)):
    #print('\nalpha = %s' %i)
        j = betas[bln]
        if j == 1:
            beta1 = bln
        
        T_expected_narrow = np.asarray([[0, 0, 2*(i+1)],
         [0, 0, 2*(j+1)],
                                       [2/(i+1), 2/(1+j), 2]])
        T_expected_broad = T_expected_narrow
        T_expected_broad_norm = np.vstack([T_expected_broad[i]/np.sum(T_expected_broad[i]) for i in range(len(T_expected_broad))])
        #print(T_expected_broad_norm, '\n\n', np.around(T_expected_broad_norm, 2))
        eigvals, eigvecs = np.linalg.eig(T_expected_broad_norm.T)
        eig_one_index = [x for x in range(len(eigvals)) if math.isclose(eigvals[x], 1.0, abs_tol = .0001)][0]
        eigvec_one = eigvecs[:, eig_one_index]
        evec_one = eigvec_one*100 / np.sum(eigvec_one)
        #print("Eigenvector for eigenvalue 1 of T_ob_norm:")
        #print(evec_one)
        #plt.plot((i, i, i), (evec_one), 'x')
        a.append(evec_one[0])
        b.append(evec_one[1])
        c.append(evec_one[2])
    x1.append(a)
    x2.append(b)
    x3.append(c)
print(alpha1,beta1)
lines = [x1,x2,x3]
vs = [v1, v2, v3]
fig,ax = plt.subplots()  
for i in range(len(lines)):
    ax.plot(alphas, [x[beta1] for x in lines[i]], c = cols[i], ls = '-', label = labels[i]) # plot all the points where beta is 1 (and alpha i sanything)
    plt.axhline(y = max(vs[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(vs[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'center right', framealpha = 0.3)
plt.xlabel('glass-glass bonding / mixed', fontsize=18)
plt.ylabel('probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for varied glass bond strength and fixed plastic bond strength")
plt.plot((1, 1,1),(100/6,100/6,400/6), 'o')
fig,ax = plt.subplots()  
for i in range(len(lines)):
    ax.plot(betas, lines[i][alpha1], c = cols[i], ls = '-', label = labels[i]) # plot all the points where beta is 1 (and alpha i sanything)
    plt.axhline(y = max(vs[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(vs[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'center right', framealpha = 0.3)
plt.xlabel('plastic-plastic bonding / mixed', fontsize=18)
plt.ylabel('probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for varied plastic bond strength and fixed  glass bond strength", fontsize=12)
plt.plot((1, 1,1),(100/6,100/6,400/6), 'o')

In [ ]:
"tESTING HYPOTHESIS WITH all position, ba"
#narrow observations array
alphas = np.linspace(0, 20, 100,endpoint =False)
betas = np.linspace(0, 20, 100,endpoint =False)
x1 = []
x2 = []
x3 = []
for aln in range(len(alphas)):
    a = []
    b = []
    c = []
    i = alphas[aln]
    if i ==1:
        alpha1 = aln
    for bln in range(len(betas)):
    #print('\nalpha = %s' %i)
        j = betas[bln]
        if j == 1:
            beta1 = bln
        
        p = (j*2/(18*(1+i+j)*(3+i))+
             4*2*(j+1)/(18*2*(3+i)*(1+i+j)*(2+i+j)))
        g = (i*2/(18*(1+i+j)*(3+j))+
             4*2*(i+1)/(18*2*(3+j)*(1+i+j)*(2+i+j)))
        m = (2*j*(i+1)/(18*(3+i)*(1+j+i))+
              8*(i+1)*(j+1)/(18*2*(3+j)*(i+j+1)*(i+j+2))+
              8*(j+1)*(i+1)/(18*2*(3+i)*(i+j+1)*(i+j+2))+
              i*2*(1+j)/(18*(3+j)*(1+i+j)))

        T_expected_narrow = np.asarray([[p, g, m],
                                        [p, g, m],
         [p, g, m]])
        T_expected_broad = T_expected_narrow
        T_expected_broad_norm = np.vstack([T_expected_broad[i]/np.sum(T_expected_broad[i]) for i in range(len(T_expected_broad))])
        #print(T_expected_broad_norm, '\n\n', np.around(T_expected_broad_norm, 2))
        eigvals, eigvecs = np.linalg.eig(T_expected_broad_norm.T)
        eig_one_index = [x for x in range(len(eigvals)) if math.isclose(eigvals[x], 1.0, abs_tol = .0001)][0]
        eigvec_one = eigvecs[:, eig_one_index]
        evec_one = eigvec_one*100 / np.sum(eigvec_one)
        #print("Eigenvector for eigenvalue 1 of T_ob_norm:")
        #print(evec_one)
        #plt.plot((i, i, i), (evec_one), 'x')
        a.append(evec_one[0])
        b.append(evec_one[1])
        c.append(evec_one[2])
    x1.append(a)
    x2.append(b)
    x3.append(c)
print(alpha1,beta1) 
lines = [x1,x2,x3]
vs = [v1, v2, v3]
fig,ax = plt.subplots()  
for i in range(len(lines)):
    ax.plot(alphas, [x[beta1] for x in lines[i]], c = cols[i], ls = '-', label = labels[i]) # plot all the points where beta is 1 (and alpha i sanything)
    plt.axhline(y = max(vs[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(vs[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'center right', framealpha = 0.3)
plt.xlabel('glass-glass bonding / mixed', fontsize=18)
plt.ylabel('probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for varied glass bond strength and fixed plastic bond strength")
plt.plot((1, 1,1),(100/6,100/6,400/6), 'o')
fig,ax = plt.subplots()  
for i in range(len(lines)):
    ax.plot(betas, lines[i][alpha1], c = cols[i], ls = '-', label = labels[i]) # plot all the points where beta is 1 (and alpha i sanything)
    plt.axhline(y = max(vs[i]), c = cols[i], ls = ':') # maximum observed transition probability across all runs
    plt.axhline(y = min(vs[i]), c = cols[i], ls = ':') # minimum observed probability to transition to state
ax.legend(loc = 'center right', framealpha = 0.3)
plt.xlabel('plastic-plastic bonding / mixed', fontsize=18)
plt.ylabel('probability of transition', fontsize=16)
fig.suptitle("Transition probabilities for varied plastic bond strength and fixed  glass bond strength", fontsize=12)
plt.plot((1, 1,1),(100/6,100/6,400/6), 'o')

In [ ]:
from matplotlib.colors import DivergingNorm

def mean(li):
    return(sum(li) / len(li))
def rang(li):
    return(abs(max(li)-min(li)))

alphas,betas = np.meshgrid(np.linspace(0,5, 50,endpoint =False),np.linspace(0,10, 100,endpoint =False))
x1 = []
x2 = []
x3 = []
tolerances = [ mean(v1),mean(v2),mean(v3)]
tols = [ rang(v1),rang(v2),rang(v3)]
def distance_from_data(alph,bet):
    T_expected_narrow = np.asarray([[0, 0, 2*(alph+1)],
     [0, 0, 2*(bet+1)],
                                   [2/(alph+1), 2/(1+bet), 2]])
    T_expected_broad = T_expected_narrow
    T_expected_broad_norm = np.vstack([T_expected_broad[i]/np.sum(T_expected_broad[i]) for i in range(len(T_expected_broad))])
    #print(T_expected_broad_norm, '\n\n', np.around(T_expected_broad_norm, 2))
    eigvals, eigvecs = np.linalg.eig(T_expected_broad_norm.T)
    eig_one_index = [x for x in range(len(eigvals)) if math.isclose(eigvals[x], 1.0, abs_tol = .0001)][0]
    eigvec_one = eigvecs[:, eig_one_index]
    evec_one = eigvec_one*100 / np.sum(eigvec_one)
        
    return(vector_dist(tolerances, evec_one) / vector_dist(tolerances, [0]*len(tolerances))) #distance between data and model, normalized to the size of the data's value
dfd = np.vectorize(distance_from_data)
dists = dfd(alphas, betas)
fig,ax = plt.subplots(figsize=(6,6))
plt.imshow(dists, cmap="cubehelix_r", norm=DivergingNorm(0)); plt.colorbar()

ax.set_aspect(1./ax.get_data_ratio())
plt.xlabel('glass/mixed', fontsize=16)
plt.ylabel('plastic/mixed', fontsize=16)
fig.suptitle("Distance of predicted steady state probability\nfrom observed transition probability\n(hinge)",fontsize=16)
plt.gca().invert_yaxis()

plt.plot(np.linspace(glas_old[0], glas_old[1], 10), np.linspace(plas_old[0], plas_old[1], 10), marker='x', c='r')

In [ ]:

from matplotlib.colors import DivergingNorm

def mean(li):
    return(sum(li) / len(li))
def rang(li):
    return(abs(max(li)-min(li)))

alphas,betas = np.meshgrid(np.linspace(0,5, 50,endpoint =False),np.linspace(0,10, 100,endpoint =False))
x1 = []
x2 = []
x3 = []
tolerances = [ mean(v1),mean(v2),mean(v3)]
tols = [ rang(v1),rang(v2),rang(v3)]
def distance_from_data(alph,bet):
    i = alph
    j = bet
    p = (j*2/(18*(1+i+j)*(3+i))+
         4*2*(j+1)/(18*2*(3+i)*(1+i+j)*(2+i+j)))
    g = (i*2/(18*(1+i+j)*(3+j))+
         4*2*(i+1)/(18*2*(3+j)*(1+i+j)*(2+i+j)))
    m = (2*j*(i+1)/(18*(3+i)*(1+j+i))+
          8*(i+1)*(j+1)/(18*2*(3+j)*(i+j+1)*(i+j+2))+
          8*(j+1)*(i+1)/(18*2*(3+i)*(i+j+1)*(i+j+2))+
          i*2*(1+j)/(18*(3+j)*(1+i+j)))

    T_expected_narrow = np.asarray([[p, g, m],
                                    [p, g, m],
     [p, g, m]])
    T_expected_broad = T_expected_narrow
    T_expected_broad_norm = np.vstack([T_expected_broad[i]/np.sum(T_expected_broad[i]) for i in range(len(T_expected_broad))])
    #print(T_expected_broad_norm, '\n\n', np.around(T_expected_broad_norm, 2))
    eigvals, eigvecs = np.linalg.eig(T_expected_broad_norm.T)
    eig_one_index = [x for x in range(len(eigvals)) if math.isclose(eigvals[x], 1.0, abs_tol = .0001)][0]
    eigvec_one = eigvecs[:, eig_one_index]
    evec_one = eigvec_one*100 / np.sum(eigvec_one)
        
    return(vector_dist(tolerances, evec_one) / vector_dist(tolerances, [0]*len(tolerances))) #distance between data and model, normalized to the size of the data's value
dfd = np.vectorize(distance_from_data)
dists = dfd(alphas, betas)
fig,ax = plt.subplots(figsize=(6,6))
plt.imshow(dists, cmap="cubehelix_r", norm=DivergingNorm(0)); plt.colorbar()

ax.set_aspect(1./ax.get_data_ratio())
plt.xlabel('glass/mixed', fontsize=16)
plt.ylabel('plastic/mixed', fontsize=16)
fig.suptitle("Distance of predicted steady state probability\nfrom observed transition probability\n(break apart)",fontsize=16)
plt.gca().invert_yaxis()

plt.plot(np.linspace(glas_old[0], glas_old[1], 10), np.linspace(plas_old[0], plas_old[1], 10), marker='x', c='r')


In [ ]:
parser_arr(glob('./wash*both.txt')[0])

In [ ]:
# GEt regime of transitions
#parser_arr(glob('./timestotrans.txt')[0])
def flattener(array_of_lists):
    return(np.array([x for y in array_of_lists for x in y]))

trans_runs= parser_arr(glob('./wash*both.txt')[0])
reduced_times = dict()
t_threshes = []
c_threshes = []    
index = list(range(len(trans_runs[0])))

vidnumset = []
print('\nlen indices', len(index))
for i in index:
    #xs_tr = []
    #xs_ti = []
    trans_matrices = []
    times_matrices = []
    N_transitions = trans_runs[0][i]
    trans_threshes = trans_runs[1][i]
    #print(np.shape(trans_runs[2]))
    transarrs = np.array(trans_runs[2][i][0])
    timarrs = np.array(trans_runs[2][i][2]) ##########
    vidnums = trans_runs[2][i][1]
    print('vidnums', vidnums)
    allowed_inds = [x for x in range(len(vidnums)) if vidnums[x] not in excludedvids]
    print('disallowed numbers', excludedvids)
    print('disallowed indices', [x for x in range(len(vidnums)) if vidnums[x] in excludedvids])
    print('allowed indices', allowed_inds)
    vidnums = [vidnums[x] for x in allowed_inds]
    print('vidnums filtered', vidnums)
    transarrs = transarrs[inds]
    timarrs = timarrs[inds]
    #print(row_norm(vidarrs))
    #vidarrs = np.cumsum(vidarrs, axis=0)
    vidindices = list(range(len(transarrs)))
    #print(vidnums)
    for vidindex in vidindices: 
        Tr_obb = transarrs[vidindex]
        trans_matrices.append(Tr_obb)
        #print('Transitions ob', Tr_obb)
        Ti_obb = timarrs[vidindex] ##################
        times_matrices.append(Ti_obb)
        #print('Times ob', Ti_obb)
    #print("\nTransitions", trans_matrices)
    #print("\n\nTimes", times_matrices)
    t_threshes.append(trans_threshes[0])
    c_threshes.append(trans_threshes[1])
    

    
    #print(len(times_matrices), len(times_matrices[0]))
    #print(np.shape(times_matrices[0][0]))
    timestotrans = np.array(times_matrices)
    print('tt_shape', np.shape(timestotrans))
    #print(type(timestotrans))
    timestotrans = np.moveaxis(timestotrans, [0,1,2,3], [3, 0,1,2])
    print('tt_shape', np.shape(timestotrans))
    #print(timestotrans[0], '\n\n',type(timestotrans[0][0][0]), timestotrans[0][0][0], [x for y in timestotrans[0][0][0] for x in y])
    indices = np.shape(timestotrans)
    #print(len(indices))
    reduced_timestotrans = np.empty(indices[0:-2], dtype = object)
    #print(reduced_timestotrans)
    for m in range(indices[0]):
        time = timestotrans[m]
        for j in range(indices[1]):
            fromrow = time[j]
            collector = []
            for k in range(indices[2]):
                tocolumn = fromrow[k]
                #print("TOCOLUMN", tocolumn)
                tocolumn = np.array(flattener(tocolumn))
                #print("flattened", tocolumn)
                
                collector.extend(tocolumn)
            if collector ==[]: #if there's nothing in a ROW
                reduced_timestotrans[m,j]= collector# [np.nan] # set the from row
            else:
                reduced_timestotrans[m,j] = collector
            #print('timesrow',reduced_timestotrans[m,j] )
    #timestotrans = np.concatenate(timestotrans, axis=-1)
    print('SHAPE', np.shape(reduced_timestotrans))
    print('reduced', reduced_timestotrans[0])
    print('len of rows per time ', [[len(i) for i in x] for x in reduced_timestotrans])
    print(len([x for y in np.ravel(timestotrans) for x in y]), len([i for x in np.ravel(reduced_timestotrans) for i in x]))
    reduced_times[i] = reduced_timestotrans
    #reduced_timestotrans = per time, pper origin, list of transitions times
    

In [ ]:

def binnerseq(seq, nbins, discardfrac):
    maximum = np.nanmax([x for y in seq for x in y])
    print('MAX', maximum)
    rang = maximum
    bins = list(np.linspace(0, np.round(maximum/discardfrac), nbins-1))
    #bins.append(maximum) 
    print('BINS', bins)
    return np.array(bins)
    

for i in index[0:1]:    
    total_time = (reduced_times[i])[0]
    stable_time= (reduced_times[i])[1]
    unstable_time= (reduced_times)[i][2]
    #print(stable_time.T)
    # for each time, separate the three axes
    times = [total_time, stable_time, unstable_time]
    print("Times: ")
    for i in times:
        print('means, ',[np.round(np.nanmean(x)) for x in i])        
        print('vaars, ',[np.round(np.nanvar(x)) for x in i])
        print(i, '\n')
    timenames = ["Frames between transitions", "Frames stable", "Frames unstable"]
    time = times[1]
    timename = timenames[1]
    #times, timenames = times[1:2], timenames[1:2]
    
    
    fig, ax = plt.subplots(figsize = (14, 10))
    fig.suptitle("Run no." + str(trans_threshes[0]) + "," + str(trans_threshes[1]))   
    ax.set_prop_cycle(color=cols)
    ax.hist(time, bins=binnerseq(time,  10, 1))######
    ax.set_xlabel(timename)
    ax.set_ylabel("Number of videos")
    ax.legend(labels)
    
    fig, ax = plt.subplots(figsize = (14, 10))
    fig.suptitle("Run no." + str(trans_threshes[0]) + "," + str(trans_threshes[1]))   
    ax.set_prop_cycle(color=cols)
    ax.hist(time, bins=binnerseq(time,  10, 6))######
    ax.set_xlabel(timename)
    ax.set_ylabel("Number of videos")
    ax.legend(labels)
    
    
    fig, ax = plt.subplots( figsize = (14, 10))
    fig.suptitle("Run no." + str(trans_threshes[0]) + "," + str(trans_threshes[1]))  
    ax.set_prop_cycle(color=cols)
    ax.boxplot(time, labels = labels)####
    ax.set_ylabel(timename)
    ax.set_xlabel("Type of contact")
    ax.legend(labels)